In [1]:
import os

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print
import dask.config
import distributed

# scipy basics
import numpy as np
import rasterio
import rasterio.features
import rasterio.transform
import rasterio.windows

from numba import jit
import concurrent.futures

import boto3
import time
import math
import ctypes
import pandas as pd

import geopandas as gpd
import pandas as pd
import rioxarray
from shapely.geometry import Polygon
import subprocess

<font size="6">Cluster management</font> 

<font size="5">Creating clusters</font> 

In [ ]:
# Full cluster
coiled_cluster = coiled.Cluster(
    n_workers=40,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "64GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

In [ ]:
# Test cluster
coiled_cluster = coiled.Cluster(
    n_workers=6,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "8GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

In [ ]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client(processes=False)
local_client

In [ ]:
local_client = Client()
local_client

In [147]:
# Local cluster with multiple workers
local_cluster = LocalCluster()  
local_client = Client(local_cluster)
local_client

/home/dagibbs22/anaconda3/envs/dask_notebook/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43745 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43745/status,
Dashboard: http://127.0.0.1:43745/status,Workers: 4
Total threads: 8,Total memory: 24.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46219,Workers: 4
Dashboard: http://127.0.0.1:43745/status,Total threads: 8
Started: Just now,Total memory: 24.91 GiB
Comm: tcp://127.0.0.1:33957,Total threads: 2
Dashboard: http://127.0.0.1:39461/status,Memory: 6.23 GiB
Nanny: tcp://127.0.0.1:46211,


<font size="5">Shutting down cloud and local clusters</font> 

In [ ]:
coiled_client.restart() 

In [51]:
coiled_cluster.shutdown()

In [ ]:
local_client.shutdown()

<font size="6">Variables</font> 

In [3]:
# General paths and constants

composite_LC_uri = 's3://gfw2-data/landcover/composite'

s3_out_dir = 'climate/AFOLU_flux_model/LULUCF/outputs'

IPCC_class_max_val = 6

# IPCC codes
forest = 1
cropland = 2
settlement = 3
wetland = 4
grassland = 5
otherland = 6

first_year = 2000
last_year = 2020

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('gfw2-data')

s3_client = boto3.client("s3")

<font size="6">General functions</font> 

In [4]:
def timestr():
    return time.strftime("%Y%m%d_%H_%M_%S")

def boundstr(bounds):
    bounds_str = "_".join([str(round(x)) for x in bounds])
    return bounds_str

def calc_chunk_length_pixels(bounds):
    chunk_length_pixels = int((bounds[3]-bounds[1]) * (40000/10))
    return chunk_length_pixels

In [5]:
# Returns list of all chunk boundaries within a bounding box for chunks of a given size
def get_chunk_bounds(chunk_params):

    min_x = chunk_params[0]
    min_y = chunk_params[1]
    max_x = chunk_params[2]
    max_y = chunk_params[3]
    chunk_size = chunk_params[4]
    
    x, y = (min_x, min_y)
    chunks = []

    # Polygon Size
    while y < max_y:
        while x < max_x:
            bounds = [
                x,
                y,
                x + chunk_size,
                y + chunk_size,
            ]
            chunks.append(bounds)
            x += chunk_size
        x = min_x
        y += chunk_size

    return chunks

In [6]:
# Returns the encompassing tile_id string in the form YYN/S_XXXE/W based on a coordinate
def xy_to_tile_id(top_left_x, top_left_y):

    lat_ceil = math.ceil(top_left_y/10.0) * 10
    lng_floor = math.floor(top_left_x/10.0) * 10
    
    lng: str = f"{str(lng_floor).zfill(3)}E" if (lng_floor >= 0) else f"{str(-lng_floor).zfill(3)}W"
    lat: str = f"{str(lat_ceil).zfill(2)}N" if (lat_ceil >= 0) else f"{str(-lat_ceil).zfill(2)}S"

    return f"{lat}_{lng}"

In [7]:
# Lazily opens tile within provided bounds (i.e. one chunk) and returns as a numpy array
# If it can't open the chunk (no data in it), it returns an array of the specified nodata value.
# TODO: It would be better if this just didn't return any array at all if there's no chunk. Returning an array of nodata is pretty inefficient.  
def get_tile_dataset_rio(uri, bounds, chunk_length_pixels, no_data_val):

    bounds_str = boundstr(bounds)

    try:
        with rasterio.open(uri) as ds:
            window = rasterio.windows.from_bounds(*bounds, ds.transform)
            data = ds.read(1, window=window)
    except:
        data = np.full((chunk_length_pixels, chunk_length_pixels), no_data_val)

    return data

In [8]:
# Prepares list of chunks to download.
# Chunks are defined by a bounding box.
def prepare_to_download_chunk(bounds, download_dict, no_data_val):
 
    futures = {}

    bounds_str = boundstr(bounds)
    tile_id = xy_to_tile_id(bounds[0], bounds[3])
    chunk_length_pixels = calc_chunk_length_pixels(bounds)

    # Submit requests to S3 for input chunks but don't actually download them yet. This queueing of the requests before downloading them speeds up the downloading
    # Approach is to download all the input chunks up front for every year to make downloading more efficient, even though it means storing more upfront
    with concurrent.futures.ThreadPoolExecutor() as executor:
        
        dask_print(f"Requesting data in chunk {bounds_str} in {tile_id}: {timestr()}")

        for key, value in download_dict.items():
            futures[executor.submit(get_tile_dataset_rio, value, bounds, chunk_length_pixels, no_data_val)] = key

    return futures

In [9]:
# Checks if tiles exist at all
def check_for_tile(download_dict):

    s3 = boto3.client('s3')

    i=0

    while i < len(list(download_dict.values())):

        s3_key = list(download_dict.values())[i][15:]

        # Breaks the loop if the tile exists
        try:
            s3.head_object(Bucket='gfw2-data', Key=s3_key)
            dask_print(f"Tile id {list(download_dict.values())[i][-12:-4]} exists. Proceeding.")
            return 1
        except:
            pass
            
        i+=1

    dask_print(f"Tile id {list(download_dict.values())[0][-12:-4]} does not exist. Skipping chunk.")
    return 0

In [10]:
# Checks whether a chunk has data in it
def check_chunk_for_data(layers, item_to_check, bounds_str, tile_id, no_data_val):

    i=0

    while i < len(list(layers.values())):

        # Checks if all the pixels have the nodata value
        min = np.min(list(layers.values())[i])  # Can't use np.all because it doesn't work in chunks that are mostly water; says nodata in chunk even if there is land

        # Breaks the loop if there is data in the chunk
        if min < no_data_val:
            dask_print(f"Data in chunk {bounds_str}. Proceeding.")
            return 1

        i+=1

    dask_print(f"No data in chunk {bounds_str} for any input.")
    return 0

In [11]:
# Saves array as a raster locally, then uploads it to s3
def save_and_upload_raster(bounds, chunk_length_pixels, tile_id, bounds_str, output_dict, is_final):

    s3_client = boto3.client("s3") # Needs to be in the same function as the upload_file call

    transform = rasterio.transform.from_bounds(*bounds, width=chunk_length_pixels, height=chunk_length_pixels)

    file_info = f'{tile_id}__{bounds_str}'

    # For every output file, saves from array to local raster, then to s3.
    # Can't save directly to s3, unfortunately, so need to save locally first.
    for key, value in output_dict.items():

        data_meaning = value[2]
        year_out = value[3]

        if not is_final:
            dask_print(f"Saving {bounds_str} in {tile_id} for {year_out}: {timestr()}")

        if is_final:
            file_name = f"{file_info}__{key}.tif"
        else:
            file_name = f"{file_info}__{key}__{timestr()}.tif"

        with rasterio.open(f"/tmp/{file_name}", 'w', driver='GTiff', width=chunk_length_pixels, height=chunk_length_pixels, count=1, dtype='uint8', crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=400, blockysize=400) as dst:
            dst.write(value[0].astype(rasterio.uint8), 1)

        s3_path = f"{s3_out_dir}/{data_meaning}/{year_out}/{chunk_length_pixels}_pixels/{time.strftime('%Y%m%d')}"

        if not is_final:
            dask_print(f"Uploading {bounds_str} in {tile_id} for {year_out} to {s3_path}: {timestr()}")

        s3_client.upload_file(f"/tmp/{file_name}", "gfw2-data", Key=f"{s3_path}/{file_name}")

        # Deletes the local raster
        os.remove(f"/tmp/{file_name}")

In [12]:
# Lists rasters in an s3 folder and returns their names as a list
def list_rasters_in_folder(full_in_folder):

    cmd = ['aws', 's3', 'ls', full_in_folder]
    s3_contents_bytes = subprocess.check_output(cmd)

    # Converts subprocess results to useful string
    s3_contents_str = s3_contents_bytes.decode('utf-8')
    s3_contents_list = s3_contents_str.splitlines()
    rasters = [line.split()[-1] for line in s3_contents_list]
    rasters = [i for i in rasters if "tif" in i]

    return rasters

In [44]:
# Uploads a shapefile to s3
def upload_shp(full_in_folder, in_folder, shp):

    dask_print(f"Uploading to {full_in_folder}{shp}: {timestr()}")

    shp_pattern = shp[:-4]

    s3_client = boto3.client("s3")  # Needs to be in the same function as the upload_file call
    s3_client.upload_file(f"/tmp/{shp}", "gfw2-data", Key=f"{in_folder[10:]}{shp}")
    s3_client.upload_file(f"/tmp/{shp_pattern}.dbf", "gfw2-data", Key=f"{in_folder[10:]}{shp_pattern}.dbf")
    s3_client.upload_file(f"/tmp/{shp_pattern}.prj", "gfw2-data", Key=f"{in_folder[10:]}{shp_pattern}.prj")
    s3_client.upload_file(f"/tmp/{shp_pattern}.shx", "gfw2-data", Key=f"{in_folder[10:]}{shp_pattern}.shx")

    os.remove(f"/tmp/{shp}")
    os.remove(f"/tmp/{shp_pattern}.dbf")
    os.remove(f"/tmp/{shp_pattern}.prj")
    os.remove(f"/tmp/{shp_pattern}.shx")

In [42]:
# Makes a shapefile of the footprints of rasters in a folder, for checking geographical completeness of rasters
def make_tile_footprint_shp(input_dict):

    in_folder = list(input_dict.keys())[0]
    pattern = list(input_dict.values())[0]

    # Task properties
    dask_print(f"Making tile index shapefile for: {in_folder}: {timestr()}")

    # Folder including s3 key
    s3_in_folder = f's3://{in_folder}'
    vsis3_in_folder = f'/vsis3/{in_folder}'

    # List of all the filenames in the folder
    filenames = list_rasters_in_folder(s3_in_folder)

    # List of the tile paths in the folder
    tile_paths = []
    tile_paths = [vsis3_in_folder + filename for filename in filenames]

    file_paths = 's3_paths.txt'

    with open(f"/tmp/{file_paths}", 'w') as file:
        for item in tile_paths:
            file.write(item + '\n')

    # Output shapefile name
    shp = f"raster_footprints_{pattern}.shp"

    cmd = ["gdaltindex", "-t_srs", "EPSG:4326", f"/tmp/{shp}", "--optfile", f"/tmp/{file_paths}"]
    subprocess.check_call(cmd)

    # Uploads shapefile to s3
    upload_shp(s3_in_folder, in_folder, shp)

    return(f"Completed: {timestr()}")

In [142]:
def flatten_list(nested_list):
    return [x for xs in nested_list for x in xs]

def create_list_for_aggregation(s3_in_folders):

    list_of_s3_name_dicts_total = []   # Final list of dictionaries of s3 paths and output aggregated 10x10 rasters
    
    # Iterates through all the desired s3 folders
    for s3_in_folder in s3_in_folders:
    
        simple_file_names = []   # List of output aggregatd 10x10 rasters
    
        # Raw filenames in a folder
        filenames = list_rasters_in_folder(f"s3://{s3_in_folder}")
    
        # Iterates through all the files in a folder and converts them to the output names. 
        # Essentially [tile_id]__[pattern].tif. Drops the chunk bounds from the middle.
        for filename in filenames:
        
            result = filename[:10] + filename[filename.rfind("__") + len("__"):]   # Extracts the relevant parts of the raw file names
            simple_file_names.append(result)   # New list of simplified file names used for 10x10 degree outputs
    
        # Removes duplicate simplified file names.
        # There are duplicates because each 10x10 output raster has many constituent chunks, each of which have the same aggregated, final name
        simple_file_names = np.unique(simple_file_names).tolist()   
    
        # Makes a list of dictionaries, where the key is the input s3 path and the value is the output aggregated name
        list_of_s3_name_dicts = [{key: value} for value in simple_file_names for key in [s3_in_folder]]
    
        # dask_print(list_of_s3_name_dicts)
    
        # Adds the dictionary of s3 paths and output names for this folder to the list for all folders
        list_of_s3_name_dicts_total.append(list_of_s3_name_dicts)
    
    # Output of above is a nested list, where each input folder is its own inner list. Need to flatten to a list.
    list_of_s3_name_dicts_total = flatten_list(list_of_s3_name_dicts_total)
    
    print(f"There are {len(list_of_s3_name_dicts_total)} chunks to process in {len(s3_in_folders)} input folders.")

    return list_of_s3_name_dicts_total

In [57]:
def merge_small_tiles(in_folder):

    s3_in_folder = f's3://{in_folder}'
    vsis3_in_folder = f'/vsis3/{in_folder}'

    filenames = list_rasters_in_folder(s3_in_folder)

    # dask_print(filenames)
    
    # List of the tile paths in the folder
    tile_paths = []
    tile_paths = [s3_in_folder + filename for filename in filenames]

    # dask_print(tile_paths)
    
    tile_string = "50N_010E"
    tile_paths = [i for i in tile_paths if tile_string in i]
    
    print(tile_paths)

    small_rasters = [rioxarray.open_rasterio(tile_path, chunks=True) for tile_path in tile_paths]

    dask_print(f"Merging {year-5}_{year} to s3")

    # merged = merge_arrays(small_rasters)  # https://corteva.github.io/rioxarray/stable/examples/merge.html

    # out_file = f'merged_change_{year-5}_{year}.tif'

    # dask_print(f"Saving locally {year-5}_{year} to s3")

    # merged.rio.to_raster(f'/tmp/{out_file}')

    # # IPCC_change_dict[f"IPCC_change_{year-5}_{year}merged"] = [merged, "uint8", "IPCC_basic_change", f'{year-5}_{year}']  

    # # save_and_upload(bounds, chunk_length_pixels, tile_id, bounds_str, IPCC_change_dict)

    # s3_client = boto3.client("s3")

    # path_and_file = f"climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/{year-5}_{year}/40000_pixels/{time.strftime('%Y%m%d')}/{out_file}"

    # dask_print(f"Uploading {year-5}_{year} to s3")

    # s3_client.upload_file(f'/tmp/{out_file}', "gfw2-data", 
    #                       Key = f'{path_and_file}')

    # dask_print(f"Done uploading {year-5}_{year} to s3")

    # os.remove(f'/tmp/{out_file}')

    # del merged

    # return f"success for {year}"

In [144]:
%%time

s3_in_folders = [
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2005/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2010/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2015/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2020/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2000_2005/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2005_2010/8000_pixels/20240205/",
           # "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2010_2015/8000_pixels/20240205/",
           "gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_change/2015_2020/8000_pixels/20240205/"
          ]

list_of_s3_name_dicts_total = create_list_for_aggregation(s3_in_folders)

# For testing. Limits the number of output rasters
list_of_s3_name_dicts_total = list_of_s3_name_dicts_total[0:3]

print(list_of_s3_name_dicts_total)



delayed_result = [dask.delayed(merge_small_tiles)(s3_name_dict) for s3_name_dict in list_of_s3_name_dicts_total]

results = dask.compute(*delayed_result)
results

There are 2520 chunks to process in 9 input folders.
[{'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_000E__IPCC_classes_2000.tif'}, {'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_010E__IPCC_classes_2000.tif'}, {'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_020E__IPCC_classes_2000.tif'}, {'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_030E__IPCC_classes_2000.tif'}, {'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_040E__IPCC_classes_2000.tif'}, {'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_040W__IPCC_classes_2000.tif'}, {'gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2000/8000_pixels/20240205/': '00N_050W__IPCC_classes_2000.t

In [102]:
strings = [
    "00N_000E__6_-2_8_0__IPCC_change_2015_2020.tif",
    "some_string__some_more_text",
    "another_example__with_more_text"
]

result_list = [string[string.rfind("__") + len("__"):] if "__" in string else "" for string in strings]

print(result_list)

['IPCC_change_2015_2020.tif', 'some_more_text', 'with_more_text']


In [93]:
string = "00N_000E__6_-2_8_0__IPCC_change_2015_2020.tif"
last_double_underscore_index = string.rfind("__")
if last_double_underscore_index != -1:  # Check if "__" was found in the string
    result = string[:10] + string[last_double_underscore_index + len("__"):]
    print(result)
else:
    print("No '__' found in the string.")

00N_000E__IPCC_change_2015_2020.tif


In [15]:
#TODO: Function to track the number of land use changes per pixel